In [ ]:
import os
import pandas as pd
from xgboost import XGBClassifier
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
)
from sklearn.model_selection import GridSearchCV
import time
import joblib

In [ ]:
X_train = pd.read_csv('../data/train_features.csv')
X_test = pd.read_csv('../data/test_features.csv')

In [3]:
X_train

,time_in_hospital,n_lab_procedures,n_procedures,n_medications,n_outpatient,n_inpatient,n_emergency,glucose_test,A1Ctest,change,...,diag_2_Other,diag_2_Respiratory,diag_3_Circulatory,diag_3_Diabetes,diag_3_Digestive,diag_3_Injury,diag_3_Missing,diag_3_Musculoskeletal,diag_3_Other,diag_3_Respiratory
0,0.615385,0.580357,0.166667,0.333333,0.000000,0.000000,0.0,0.0,1.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.076923,0.339286,0.166667,0.115385,0.000000,0.066667,0.0,0.0,0.0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.196429,0.000000,0.025641,0.000000,0.000000,0.0,0.0,2.0,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.307692,0.473214,0.666667,0.256410,0.000000,0.000000,0.0,0.0,1.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.230769,0.473214,0.000000,0.102564,0.000000,0.066667,0.0,0.0,2.0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17495,0.461538,0.383929,0.000000,0.333333,0.148148,0.200000,0.0,0.0,0.0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
17496,0.230769,0.544643,0.000000,0.410256,0.000000,0.133333,0.0,0.0,0.0,1,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
17497,0.153846,0.107143,0.166667,0.397436,0.000000,0.000000,0.0,0.0,0.0,1,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
17498,0.538462,0.508929,0.666667,0.217949,0.000000,0.000000,0.0,0.0,2.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [4]:
X_test

,time_in_hospital,n_lab_procedures,n_procedures,n_medications,n_outpatient,n_inpatient,n_emergency,glucose_test,A1Ctest,change,...,diag_2_Other,diag_2_Respiratory,diag_3_Circulatory,diag_3_Diabetes,diag_3_Digestive,diag_3_Injury,diag_3_Missing,diag_3_Musculoskeletal,diag_3_Other,diag_3_Respiratory
0,0.000000,0.303571,0.000000,0.076923,0.037037,0.133333,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.076923,0.080357,1.000000,0.217949,0.000000,0.000000,0.0,0.0,0.0,0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.846154,0.517857,0.166667,0.179487,0.000000,0.000000,0.0,0.0,0.0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.076923,0.276786,0.000000,0.230769,0.000000,0.000000,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.000000,0.419643,0.000000,0.217949,0.000000,0.000000,0.0,0.0,0.0,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,0.230769,0.303571,0.000000,0.179487,0.000000,0.000000,0.0,0.0,0.0,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7496,0.307692,0.348214,0.166667,0.141026,0.000000,0.000000,0.0,0.0,0.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7497,0.230769,0.410714,0.500000,0.256410,0.000000,0.133333,0.0,0.0,0.0,0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7498,0.153846,0.339286,0.000000,0.141026,0.000000,0.000000,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
y_train = pd.read_csv('../data/train_labels.csv').squeeze()
y_test = pd.read_csv('../data/test_labels.csv').squeeze()

In [6]:
y_train

0        0
1        1
2        0
3        0
4        0
        ..
17495    0
17496    0
17497    0
17498    0
17499    1
Name: readmission_status, Length: 17500, dtype: int64

In [7]:
y_test

0       1
1       0
2       0
3       0
4       1
       ..
7495    0
7496    1
7497    1
7498    1
7499    0
Name: readmission_status, Length: 7500, dtype: int64

In [8]:
model = XGBClassifier(
    use_label_encoder=False, 
    eval_metric='auc',
    objective='binary:logistic',
    random_state=42
)

In [9]:
X_train.columns

Index(['time_in_hospital', 'n_lab_procedures', 'n_procedures', 'n_medications',
       'n_outpatient', 'n_inpatient', 'n_emergency', 'glucose_test', 'A1Ctest',
       'change', 'diabetes_med', 'age_[40-50)', 'age_[50-60)', 'age_[60-70)',
       'age_[70-80)', 'age_[80-90)', 'age_[90-100)',
       'medical_specialty_Cardiology', 'medical_specialty_Emergency/Trauma',
       'medical_specialty_Family/GeneralPractice',
       'medical_specialty_InternalMedicine', 'medical_specialty_Missing',
       'medical_specialty_Other', 'medical_specialty_Surgery',
       'diag_1_Circulatory', 'diag_1_Diabetes', 'diag_1_Digestive',
       'diag_1_Injury', 'diag_1_Missing', 'diag_1_Musculoskeletal',
       'diag_1_Other', 'diag_1_Respiratory', 'diag_2_Circulatory',
       'diag_2_Diabetes', 'diag_2_Digestive', 'diag_2_Injury',
       'diag_2_Missing', 'diag_2_Musculoskeletal', 'diag_2_Other',
       'diag_2_Respiratory', 'diag_3_Circulatory', 'diag_3_Diabetes',
       'diag_3_Digestive', 'diag_3_Injury

In [10]:
def sanitize_column_names(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('[', '').str.replace(')', '')
    return df

In [11]:
sanitize_column_names(X_train)
sanitize_column_names(X_test)

,time_in_hospital,n_lab_procedures,n_procedures,n_medications,n_outpatient,n_inpatient,n_emergency,glucose_test,a1ctest,change,...,diag_2_other,diag_2_respiratory,diag_3_circulatory,diag_3_diabetes,diag_3_digestive,diag_3_injury,diag_3_missing,diag_3_musculoskeletal,diag_3_other,diag_3_respiratory
0,0.000000,0.303571,0.000000,0.076923,0.037037,0.133333,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.076923,0.080357,1.000000,0.217949,0.000000,0.000000,0.0,0.0,0.0,0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.846154,0.517857,0.166667,0.179487,0.000000,0.000000,0.0,0.0,0.0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.076923,0.276786,0.000000,0.230769,0.000000,0.000000,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.000000,0.419643,0.000000,0.217949,0.000000,0.000000,0.0,0.0,0.0,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,0.230769,0.303571,0.000000,0.179487,0.000000,0.000000,0.0,0.0,0.0,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7496,0.307692,0.348214,0.166667,0.141026,0.000000,0.000000,0.0,0.0,0.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7497,0.230769,0.410714,0.500000,0.256410,0.000000,0.133333,0.0,0.0,0.0,0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7498,0.153846,0.339286,0.000000,0.141026,0.000000,0.000000,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [12]:
X_train.columns

Index(['time_in_hospital', 'n_lab_procedures', 'n_procedures', 'n_medications',
       'n_outpatient', 'n_inpatient', 'n_emergency', 'glucose_test', 'a1ctest',
       'change', 'diabetes_med', 'age_40-50', 'age_50-60', 'age_60-70',
       'age_70-80', 'age_80-90', 'age_90-100', 'medical_specialty_cardiology',
       'medical_specialty_emergency/trauma',
       'medical_specialty_family/generalpractice',
       'medical_specialty_internalmedicine', 'medical_specialty_missing',
       'medical_specialty_other', 'medical_specialty_surgery',
       'diag_1_circulatory', 'diag_1_diabetes', 'diag_1_digestive',
       'diag_1_injury', 'diag_1_missing', 'diag_1_musculoskeletal',
       'diag_1_other', 'diag_1_respiratory', 'diag_2_circulatory',
       'diag_2_diabetes', 'diag_2_digestive', 'diag_2_injury',
       'diag_2_missing', 'diag_2_musculoskeletal', 'diag_2_other',
       'diag_2_respiratory', 'diag_3_circulatory', 'diag_3_diabetes',
       'diag_3_digestive', 'diag_3_injury', 'diag_3_m

In [13]:
X_test.columns

Index(['time_in_hospital', 'n_lab_procedures', 'n_procedures', 'n_medications',
       'n_outpatient', 'n_inpatient', 'n_emergency', 'glucose_test', 'a1ctest',
       'change', 'diabetes_med', 'age_40-50', 'age_50-60', 'age_60-70',
       'age_70-80', 'age_80-90', 'age_90-100', 'medical_specialty_cardiology',
       'medical_specialty_emergency/trauma',
       'medical_specialty_family/generalpractice',
       'medical_specialty_internalmedicine', 'medical_specialty_missing',
       'medical_specialty_other', 'medical_specialty_surgery',
       'diag_1_circulatory', 'diag_1_diabetes', 'diag_1_digestive',
       'diag_1_injury', 'diag_1_missing', 'diag_1_musculoskeletal',
       'diag_1_other', 'diag_1_respiratory', 'diag_2_circulatory',
       'diag_2_diabetes', 'diag_2_digestive', 'diag_2_injury',
       'diag_2_missing', 'diag_2_musculoskeletal', 'diag_2_other',
       'diag_2_respiratory', 'diag_3_circulatory', 'diag_3_diabetes',
       'diag_3_digestive', 'diag_3_injury', 'diag_3_m

In [14]:
print("Training XGBoost model...")
model.fit(X_train, y_train)
print("Training complete.")

Training XGBoost model...


/media/anubhaw/Anubhaw/Projects/Hospital Readmission Prediction/myenv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [03:14:06] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Training complete.


In [15]:
y_pred_class = model.predict(X_test)

In [16]:
y_pred_class

array([0, 0, 0, ..., 1, 0, 1], shape=(7500,))

In [17]:
y_pred_class

array([0, 0, 0, ..., 1, 0, 1], shape=(7500,))

In [18]:
y_pred_proba = model.predict_proba(X_test)[:, 1]

In [19]:
y_pred_proba

array([0.4108077 , 0.35813728, 0.4086698 , ..., 0.63807845, 0.351189  ,
       0.6379881 ], shape=(7500,), dtype=float32)

In [20]:
def evaluate_model(model):
    y_pred_class = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    accuracy = accuracy_score(y_test, y_pred_class)
    precision = precision_score(y_test, y_pred_class)
    recall = recall_score(y_test, y_pred_class)
    f1 = f1_score(y_test, y_pred_class)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    cm = confusion_matrix(y_test, y_pred_class)
    cm_df = pd.DataFrame(
        cm,
        index=['Actual No Readmission (0)', 'Actual Readmission (1)'],
        columns=['Predicted No Readmission (0)', 'Predicted Readmission (1)']
    )
    
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall (Sensitivity): {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"ROC-AUC: {roc_auc:.4f}")
    print("Confusion Matrix:")
    print(cm_df)
    print("\nInterpretation of Confusion Matrix:")
    print(f"  True Negatives (TN): {cm[0, 0]} (Correctly predicted NOT readmitted)")
    print(f"  False Positives (FP): {cm[0, 1]} (Incorrectly predicted readmitted - Resource Waste)")
    print(f"  False Negatives (FN): {cm[1, 0]} (Incorrectly predicted NOT readmitted - Missed Intervention)")
    print(f"  True Positives (TP): {cm[1, 1]} (Correctly predicted readmitted)")

In [21]:
evaluate_model(model)

Accuracy: 0.6045
Precision: 0.5851
Recall (Sensitivity): 0.5246
F1 Score: 0.5532
ROC-AUC: 0.6358
Confusion Matrix:
                           Predicted No Readmission (0)  \
Actual No Readmission (0)                          2698   
Actual Readmission (1)                             1664   

                           Predicted Readmission (1)  
Actual No Readmission (0)                       1302  
Actual Readmission (1)                          1836  

Interpretation of Confusion Matrix:
  True Negatives (TN): 2698 (Correctly predicted NOT readmitted)
  False Positives (FP): 1302 (Incorrectly predicted readmitted - Resource Waste)
  False Negatives (FN): 1664 (Incorrectly predicted NOT readmitted - Missed Intervention)
  True Positives (TP): 1836 (Correctly predicted readmitted)


In [22]:
y_train[y_train == 0].count() / y_train.count()

np.float64(0.5283428571428571)

In [23]:
y_train[y_train == 1].count() / y_train.count()

np.float64(0.47165714285714283)

In [24]:
neg_count = np.sum(y_train == 0)
pos_count = np.sum(y_train == 1)
scale_pos_weight_value = neg_count / pos_count
print(f"Calculated scale_pos_weight: {scale_pos_weight_value:.2f}")

Calculated scale_pos_weight: 1.12


In [31]:
param_grid = {
    # Number of boosting rounds/trees (Start low, tune later)
    'n_estimators': [100, 200, 300], 
    
    # Max depth of each tree (Controls complexity/overfitting)
    'max_depth': [3, 5, 7], 
    
    # Learning rate (Controls step size/how fast the model learns)
    'learning_rate': [0.01, 0.05, 0.1], 
    
    # Set the calculated weight for the positive class (readmission)
    'scale_pos_weight': [scale_pos_weight_value] 
}

In [32]:
start_time = time.time()
print("\nStarting GridSearchCV...")

gscv = GridSearchCV(
    # Base Estimator: XGBoost Classifier (with fixed parameters)
    estimator=XGBClassifier(
        objective='binary:logistic', 
        eval_metric='auc',
        use_label_encoder=False,
        random_state=42
    ),
    
    # Search Space: The dictionary of hyperparameters
    param_grid=param_grid,
    
    # Scoring Metric: Use the AUC-ROC scorer defined above
    scoring='roc_auc',
    
    # Cross-Validation Folds: 5-fold is standard
    cv=5,
    
    # Verbosity: Shows progress during the search
    verbose=3,
    
    # Number of cores to use: -1 uses all available cores for speed
    n_jobs=-1
)

# This step trains hundreds of models based on the combinations in param_grid
gscv.fit(X_train, y_train)

end_time = time.time()
print(f"\nGrid Search completed in {(end_time - start_time) / 60:.2f} minutes.")


Starting GridSearchCV...
Fitting 5 folds for each of 27 candidates, totalling 135 fits


/media/anubhaw/Anubhaw/Projects/Hospital Readmission Prediction/myenv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [03:19:26] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/media/anubhaw/Anubhaw/Projects/Hospital Readmission Prediction/myenv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [03:19:26] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/media/anubhaw/Anubhaw/Projects/Hospital Readmission Prediction/myenv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [03:19:26] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/media/anubhaw/Anubhaw/Projects/Hospital Readmission Prediction/myenv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [03:19:26] 

[CV 1/5] END learning_rate=0.01, max_depth=3, n_estimators=100, scale_pos_weight=1.1201841531378725;, score=0.636 total time=   0.6s
[CV 3/5] END learning_rate=0.01, max_depth=3, n_estimators=100, scale_pos_weight=1.1201841531378725;, score=0.639 total time=   0.7s
[CV 2/5] END learning_rate=0.01, max_depth=3, n_estimators=100, scale_pos_weight=1.1201841531378725;, score=0.641 total time=   0.7s
[CV 4/5] END learning_rate=0.01, max_depth=3, n_estimators=100, scale_pos_weight=1.1201841531378725;, score=0.643 total time=   0.7s


/media/anubhaw/Anubhaw/Projects/Hospital Readmission Prediction/myenv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [03:19:27] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/media/anubhaw/Anubhaw/Projects/Hospital Readmission Prediction/myenv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [03:19:27] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/media/anubhaw/Anubhaw/Projects/Hospital Readmission Prediction/myenv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [03:19:27] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/media/anubhaw/Anubhaw/Projects/Hospital Readmission Prediction/myenv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [03:19:27] 

[CV 5/5] END learning_rate=0.01, max_depth=3, n_estimators=100, scale_pos_weight=1.1201841531378725;, score=0.643 total time=   0.8s


...

[CV 2/5] END learning_rate=0.1, max_depth=7, n_estimators=200, scale_pos_weight=1.1201841531378725;, score=0.634 total time=   1.7s


/media/anubhaw/Anubhaw/Projects/Hospital Readmission Prediction/myenv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [03:19:48] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[CV 4/5] END learning_rate=0.1, max_depth=7, n_estimators=200, scale_pos_weight=1.1201841531378725;, score=0.635 total time=   1.5s
[CV 3/5] END learning_rate=0.1, max_depth=7, n_estimators=200, scale_pos_weight=1.1201841531378725;, score=0.633 total time=   1.6s
[CV 5/5] END learning_rate=0.1, max_depth=7, n_estimators=200, scale_pos_weight=1.1201841531378725;, score=0.635 total time=   1.6s
[CV 1/5] END learning_rate=0.1, max_depth=7, n_estimators=300, scale_pos_weight=1.1201841531378725;, score=0.634 total time=   1.9s
[CV 3/5] END learning_rate=0.1, max_depth=7, n_estimators=300, scale_pos_weight=1.1201841531378725;, score=0.626 total time=   1.9s
[CV 2/5] END learning_rate=0.1, max_depth=7, n_estimators=300, scale_pos_weight=1.1201841531378725;, score=0.629 total time=   2.1s
[CV 4/5] END learning_rate=0.1, max_depth=7, n_estimators=300, scale_pos_weight=1.1201841531378725;, score=0.628 total time=   1.4s
[CV 5/5] END learning_rate=0.1, max_depth=7, n_estimators=300, scale_pos_wei

/media/anubhaw/Anubhaw/Projects/Hospital Readmission Prediction/myenv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [03:19:50] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



Grid Search completed in 0.40 minutes.


In [33]:
print("\n--- Grid Search Results ---")
print(f"Best AUC-ROC Score achieved: {gscv.best_score_:.4f}")
print("Best Hyperparameters found:")
print(gscv.best_params_)

# Retrieve the best model object
best_xgb_model = gscv.best_estimator_


--- Grid Search Results ---
Best AUC-ROC Score achieved: 0.6569
Best Hyperparameters found:
{'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 300, 'scale_pos_weight': np.float64(1.1201841531378725)}


Testing the best fit model on test data

In [34]:
y_test_proba = best_xgb_model.predict_proba(X_test)[:, 1]
y_test_pred_class = best_xgb_model.predict(X_test)

In [ ]:
print('\nBest XGBoost Model Evaluation on Test Set:')
evaluate_model(best_xgb_model)

Accuracy: 0.6181
Precision: 0.5984
Recall (Sensitivity): 0.5526
F1 Score: 0.5746
ROC-AUC: 0.6573
Confusion Matrix:
                           Predicted No Readmission (0)  \
Actual No Readmission (0)                          2702   
Actual Readmission (1)                             1566   

                           Predicted Readmission (1)  
Actual No Readmission (0)                       1298  
Actual Readmission (1)                          1934  

Interpretation of Confusion Matrix:
  True Negatives (TN): 2702 (Correctly predicted NOT readmitted)
  False Positives (FP): 1298 (Incorrectly predicted readmitted - Resource Waste)
  False Negatives (FN): 1566 (Incorrectly predicted NOT readmitted - Missed Intervention)
  True Positives (TP): 1934 (Correctly predicted readmitted)


### Saving the model

In [ ]:
try:
    os.makedirs('../models', exist_ok=True)
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
joblib.dump(best_xgb_model, '../models/best_xgb_model.joblib')

['model/best_xgb_model.joblib']